In [2]:
import requests
import pathlib

# 確認 data 資料夾是否存在
pathlib.Path('data').mkdir(exist_ok=True)

# 楓葉紅葉日期
response = requests.get('https://www.data.jma.go.jp/sakura/ruinenchi/015.csv')
response.encoding = 'shift-jis'
with open('data/maple_foliage.csv', 'w', encoding='utf-8') as file:
    file.write(response.text)

# 楓葉落葉日期
response = requests.get('https://www.data.jma.go.jp/sakura/ruinenchi/016.csv')
response.encoding = 'shift-jis'
with open('data/maple_shedding.csv', 'w', encoding='utf-8') as file:
    file.write(response.text)

In [3]:
import pandas as pd

foliage_data = pd.read_csv('data/maple_foliage.csv', skiprows=1, encoding='utf-8')
shedding_data = pd.read_csv('data/maple_shedding.csv', skiprows=1, encoding='utf-8')

foliage_data.head()

,番号,地点名,1953,rm,1954,rm.1,1955,rm.2,1956,rm.3,...,2032,rm.79,平年値,rm.80,最早値,rm.81,最早年,最晩値,rm.82,最晩年
0,401,稚内,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,406,留萌,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,407,旭川,0,0,0,0,0,0,0,0,...,0,0,1023,6,1008,6,1993,1105,6,2002
3,409,網走,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,412,札幌,1008,6,1015,6,1017,6,1024,6,...,0,0,1028,6,926,6,1963,1110,6,2023


In [4]:
def drop_unused_columns(df):
    columns = ['番号', '平年値', '最早値', '最早年', '最晩値', '最晩年']
    remark_columns = [col for col in df.columns if col.startswith('rm')]
    return df.drop(columns=columns + remark_columns)

foliage_data = drop_unused_columns(foliage_data)
shedding_data = drop_unused_columns(shedding_data)

foliage_data.head()


,地点名,1953,1954,1955,1956,1957,1958,1959,1960,1961,...,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032
0,稚内,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,留萌,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,旭川,0,0,0,0,0,0,0,0,0,...,1027,0,0,0,0,0,0,0,0,0
3,網走,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,札幌,1008,1015,1017,1024,1020,1012,0,0,1013,...,1110,0,0,0,0,0,0,0,0,0


In [5]:
foliage_data['地点名'] = foliage_data['地点名'].str.strip()
locations = foliage_data['地点名'].unique()
locations

array(['稚内', '留萌', '旭川', '網走', '札幌', '岩見沢', '帯広', '釧路', '根室', '室蘭', '浦河',
       '江差', '函館', '倶知安', '紋別', '広尾', '新庄', '青森', '八戸', '秋田', '盛岡', '宮古',
       '酒田', '山形', '仙台', '福島', '白河', '小名浜', '輪島', '相川', '新潟', '金沢', '富山',
       '長野', '高田', '宇都宮', '福井', '高山', '松本', '前橋', '熊谷', '水戸', '敦賀', '岐阜',
       '名古屋', '飯田', '甲府', '銚子', '津', '浜松', '静岡', '東京', '尾鷲', '横浜', '館山',
       '大島', '三宅島', '八丈島', '西郷', '松江', '米子', '鳥取', '豊岡', '舞鶴', '浜田', '京都',
       '彦根', '下関', '広島', '岡山', '神戸', '大阪', '洲本', '和歌山', '潮岬', '奈良', '厳原',
       '福岡', '佐賀', '大分', '長崎', '熊本', '延岡', '鹿児島', '宮崎', '屋久島', '種子島',
       '福江', '松山', '高松', '宇和島', '高知', '徳島', '名瀬', '与那国島', '西表島', '石垣島',
       '宮古島', '久米島', '那覇', '名護', '南大東島'], dtype=object)

In [8]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="kaede")
location = geolocator.geocode(locations[0])
location.latitude, location.longitude

(45.4166166, 141.6771991)